In [ ]:
# !pip install geemap

In [ ]:
import ee
import geemap

### Create an interactive map

In [ ]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

### Add data to the map

In [ ]:
point = ee.Geometry.Point([-78.556023, 44.201128])
# point = ee.Geometry.Point([-87.7719, 41.8799])

image = (
    ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
    .filterBounds(point)
    .filterDate('2020-01-01', '2020-12-31')
    .sort('CLOUD_COVER')
    .first()
    .select('B[1-7]')
)

vis_params = {'min': 0, 'max': 3000, 'bands': ['B5', 'B4', 'B3']}

Map.centerObject(point, 8)
Map.addLayer(image, vis_params, "Landsat-8")

### Check image properties

In [ ]:
ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()

'2020-01-17'

In [ ]:
image.get('CLOUD_COVER').getInfo()

0.24

### Make training dataset

There are several ways you can create a region for generating the training dataset.

- Draw a shape (e.g., rectangle) on the map and the use `region = Map.user_roi`
- Define a geometry, such as `region = ee.Geometry.Rectangle([-122.6003, 37.4831, -121.8036, 37.8288])`
- Create a buffer zone around a point, such as `region = ee.Geometry.Point([-122.4439, 37.7538]).buffer(10000)`
- If you don't define a region, it will use the image footprint by default

In [ ]:
# region = Map.user_roi
# region = ee.Geometry.Rectangle([-122.6003, 37.4831, -121.8036, 37.8288])
# region = ee.Geometry.Point([-122.4439, 37.7538]).buffer(10000)

use the [Canada AAFC Annual Crop Inventory](https://developers.google.com/earth-engine/datasets/catalog/AAFC_ACI#bands) to create label dataset for training


![](https://www.researchgate.net/profile/Meisam-Amani/publication/345082789/figure/fig1/AS:952723008651265@1604158274050/Annual-Space-based-Crop-Inventory-ACI-produced-by-Agriculture-and-Agri-Food-Canada.png)


In [ ]:
# nlcd = ee.Image('USGS/NLCD/NLCD2016').select('landcover').clip(image.geometry())

affc = ee.ImageCollection('AAFC/ACI').filter(ee.Filter.date('2020-01-01', '2020-12-31')).first().clip(image.geometry())

Map.addLayer(affc, {}, 'AAFC')
Map

Map(center=[44.201128, -78.556023], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBo…

In [ ]:
# Make the training dataset.
points = affc.sample(
    **{
        'region': image.geometry(),
        'scale': 30,
        'numPixels': 5000,
        'seed': 0,
        'geometries': True,  # Set this to False to ignore geometries
    }
)

Map.addLayer(points, {}, 'training', False)

In [ ]:
print(points.size().getInfo())

4934


In [ ]:
print(points.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-77.2002903987127, 44.762894951029]}, 'id': '0', 'properties': {'landcover': 230}}


### Train the classifier

In [ ]:
# Use these bands for prediction.
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']


# This property of the table stores the land cover labels.
label = 'landcover'

# Overlay the points on the imagery to get training.
training = image.select(bands).sampleRegions(
    **{'collection': points, 'properties': [label], 'scale': 30}
)

# Train a CART classifier with default parameters.
trained = ee.Classifier.smileCart().train(training, label, bands)

In [ ]:
print(training.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'B1': -303, 'B2': -286, 'B3': 169, 'B4': 49, 'B5': 2741, 'B6': 536, 'B7': 434, 'landcover': 230}}


### Classify the image

In [ ]:
# Classify the image with the same bands used for training.
result = image.select(bands).classify(trained)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'classified')
Map

Map(center=[44.201128, -78.556023], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBo…

### Render categorical map

To render a categorical map, we can set two image properties: `landcover_class_values` and `landcover_class_palette`. We can use the same style as the NLCD so that it is easy to compare the two maps. 

In [ ]:
class_values = affc.get('landcover_class_values').getInfo()
class_values

[10,
 20,
 30,
 34,
 35,
 50,
 80,
 85,
 110,
 120,
 122,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 160,
 161,
 162,
 163,
 167,
 168,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 185,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 210,
 220,
 230]

In [ ]:
class_palette = affc.get('landcover_class_palette').getInfo()
class_palette

['000000',
 '3333ff',
 '996666',
 'cc6699',
 'e1e1e1',
 'ffff00',
 '993399',
 '501b50',
 'cccc00',
 'cc6600',
 'ffcc33',
 '7899f6',
 'ff9900',
 '660000',
 'dae31d',
 'd6cc00',
 'd2db25',
 'd1d52b',
 'cace32',
 'c3c63a',
 'b9bc44',
 'a7b34d',
 'b9c64e',
 '999900',
 'e9e2b1',
 '92a55b',
 '809769',
 'ffff99',
 '98887c',
 '799b93',
 '5ea263',
 '52ae77',
 '41bf7a',
 'd6ff70',
 '8c8cff',
 'd6cc00',
 'ff7f00',
 '315491',
 'cc9933',
 '896e43',
 '996633',
 '8f6c3d',
 'b6a472',
 '82654a',
 'a39069',
 'b85900',
 'b74b15',
 'ff8a8a',
 'ffcccc',
 '6f55ca',
 'ffccff',
 'dc5424',
 'd05a30',
 'd20000',
 'cc0000',
 'dc3200',
 'ff6666',
 'c5453b',
 '7442bd',
 'ffcccc',
 'b5fb05',
 'ccff05',
 '07f98c',
 '00ffcc',
 'cc33cc',
 '8e7672',
 'b1954f',
 '749a66',
 '009900',
 '006600',
 '00cc00',
 'cc9900']

In [ ]:
landcover = result.set('classification_class_values', class_values)
landcover = landcover.set('classification_class_palette', class_palette)

In [ ]:
Map.addLayer(landcover, {}, 'Land cover')
Map

Map(center=[44.201128, -78.556023], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBo…

### Visualize the result

In [ ]:
print('Change layer opacity:')
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

Change layer opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

### Add a legend to the map

In [ ]:
# Map.add_legend(builtin_legend='NLCD')
# Map

### Export the result

Export the result directly to your computer:

In [ ]:
# import os

# out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
# out_file = os.path.join(out_dir, 'landcover.tif')

In [ ]:
# geemap.ee_export_image(landcover, filename=out_file, scale=900)

Export the result to Google Drive:

In [ ]:
# geemap.ee_export_image_to_drive(
#     landcover, description='landcover', folder='export', scale=900
# )